<a href="https://colab.research.google.com/github/JSGCunha/JSGCunha/blob/main/Case1_peers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=c4de60749d0a7d6f29033ac43f6ac2da8bf18c3b9ba90042d5b9ef20db4ab593
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [4]:
import pandas as pd
from fpdf import FPDF

# Nome do arquivo Excel
excel_file_name = 'Case_Infomaz_Base_de_Dados.xlsx'
output_pdf = 'relatorio_resultados.pdf'

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'Relatório de Resultados - Case Infomaz', ln=True, align='C')
        self.ln(5)

    def add_table(self, title, df):
        self.set_font('Arial', 'B', 11)
        self.cell(0, 10, title, ln=True)
        self.set_font('Arial', '', 9)
        for row in df.itertuples(index=False):
            row_text = ' | '.join(str(item) for item in row)
            self.multi_cell(0, 5, row_text)
        self.ln(5)

pdf = PDF()
pdf.add_page()

# Carregar dados
excel_file = pd.ExcelFile(excel_file_name)
dataframes = {sheet: excel_file.parse(sheet, header=1) for sheet in excel_file.sheet_names}

# Pré-processamento
for df in dataframes.values():
    df.columns = df.columns.astype(str).str.strip().str.replace('\n', '', regex=False)

def conv_num(col):
    return pd.to_numeric(col.astype(str).str.replace(',', '.', regex=False), errors='coerce')

transacoes_vendas_df = dataframes.get('Transações Vendas')
cadastro_produtos_df = dataframes.get('Cadastro Produtos')
cadastro_estoque_df = dataframes.get('Cadastro de Estoque')
cadastro_clientes_df = dataframes.get('Cadastro Clientes')
cadastro_fornecedores_df = dataframes.get('Cadastro Fornecedores')

transacoes_vendas_df['VALOR ITEM'] = conv_num(transacoes_vendas_df['VALOR ITEM'])
transacoes_vendas_df['QTD ITEM'] = pd.to_numeric(transacoes_vendas_df['QTD ITEM'], errors='coerce')
transacoes_vendas_df['DATA NOTA'] = pd.to_datetime(transacoes_vendas_df['DATA NOTA'], errors='coerce')
transacoes_vendas_df['ANO_MES'] = transacoes_vendas_df['DATA NOTA'].dt.to_period('M')

cadastro_estoque_df['VALOR ESTOQUE'] = conv_num(cadastro_estoque_df['VALOR ESTOQUE']) / 100
cadastro_estoque_df['QTD ESTOQUE'] = pd.to_numeric(cadastro_estoque_df['QTD ESTOQUE'], errors='coerce')
cadastro_estoque_df['DATA ESTOQUE'] = pd.to_datetime(cadastro_estoque_df['DATA ESTOQUE'], errors='coerce')
cadastro_estoque_df['ANO_MES'] = cadastro_estoque_df['DATA ESTOQUE'].dt.to_period('M')

# Pergunta 2
merged = pd.merge(transacoes_vendas_df, cadastro_produtos_df, on='ID PRODUTO', how='left')
merged = pd.merge(merged, cadastro_estoque_df[['ID ESTOQUE', 'VALOR ESTOQUE']], on='ID ESTOQUE', how='left')
merged.dropna(subset=['VALOR ITEM', 'VALOR ESTOQUE', 'QTD ITEM'], inplace=True)
merged['MARGEM_ITEM_UNITARIO'] = merged['VALOR ITEM'] - merged['VALOR ESTOQUE']
merged['TOTAL_MARGEM_ITEM'] = merged['MARGEM_ITEM_UNITARIO'] * merged['QTD ITEM']
pdf.add_table("Margem Média por Produto", merged.groupby(['ID PRODUTO', 'NOME PRODUTO'])['MARGEM_ITEM_UNITARIO'].mean().reset_index())
merged['CATEGORIA'] = merged['CATEGORIA'].fillna('Sem Categoria')
pdf.add_table("Margem Total por Categoria", merged.groupby('CATEGORIA')['TOTAL_MARGEM_ITEM'].sum().reset_index())

# Pergunta 3
cliente_mes = transacoes_vendas_df.groupby(['ID CLIENTE', 'ANO_MES'])['QTD ITEM'].sum().reset_index()
cliente_mes = pd.merge(cliente_mes, cadastro_clientes_df, on='ID CLIENTE', how='left')
cliente_mes['NOME CLIENTE'] = cliente_mes['NOME CLIENTE'].fillna(cliente_mes['ID CLIENTE'].apply(lambda x: f"Cliente ID {x}"))
pdf.add_table("Ranking Clientes por QTD/Mês", cliente_mes)

# Pergunta 4
fornecedor_mes = cadastro_estoque_df.groupby(['ID FORNECEDOR', 'ANO_MES'])['QTD ESTOQUE'].sum().reset_index()
fornecedor_mes = pd.merge(fornecedor_mes, cadastro_fornecedores_df, on='ID FORNECEDOR', how='left')
fornecedor_mes['NOME FORNECEDOR'] = fornecedor_mes['NOME FORNECEDOR'].fillna(fornecedor_mes['ID FORNECEDOR'].apply(lambda x: f"Fornecedor ID {x}"))
pdf.add_table("Ranking Fornecedores por Estoque/Mês", fornecedor_mes)

# Pergunta 5
qtd_mes = transacoes_vendas_df.groupby(['ID PRODUTO', 'ANO_MES'])['QTD ITEM'].sum().reset_index()
qtd_mes = pd.merge(qtd_mes, cadastro_produtos_df[['ID PRODUTO', 'NOME PRODUTO']], on='ID PRODUTO', how='left')
pdf.add_table("Ranking Produtos por QTD/Mês", qtd_mes)

# Pergunta 6
transacoes_vendas_df['TOTAL VENDA ITEM'] = transacoes_vendas_df['VALOR ITEM'] * transacoes_vendas_df['QTD ITEM']
valor_mes = transacoes_vendas_df.groupby(['ID PRODUTO', 'ANO_MES'])['TOTAL VENDA ITEM'].sum().reset_index()
valor_mes = pd.merge(valor_mes, cadastro_produtos_df[['ID PRODUTO', 'NOME PRODUTO']], on='ID PRODUTO', how='left')
pdf.add_table("Ranking Produtos por Valor Venda/Mês", valor_mes)

# Pergunta 7
categoria = pd.merge(transacoes_vendas_df[['ID PRODUTO', 'TOTAL VENDA ITEM', 'ANO_MES']], cadastro_produtos_df[['ID PRODUTO', 'CATEGORIA']], on='ID PRODUTO', how='left')
categoria['CATEGORIA'] = categoria['CATEGORIA'].fillna('Sem Categoria')
pdf.add_table("Média Valor Venda por Categoria/Mês", categoria.groupby(['CATEGORIA', 'ANO_MES'])['TOTAL VENDA ITEM'].mean().reset_index())

# Pergunta 8
lucro = pd.merge(transacoes_vendas_df[['ID PRODUTO', 'VALOR ITEM', 'QTD ITEM']], cadastro_produtos_df[['ID PRODUTO', 'ID ESTOQUE', 'CATEGORIA']], on='ID PRODUTO', how='left')
lucro = pd.merge(lucro, cadastro_estoque_df[['ID ESTOQUE', 'VALOR ESTOQUE']], on='ID ESTOQUE', how='left')
lucro.dropna(subset=['VALOR ITEM', 'VALOR ESTOQUE', 'QTD ITEM'], inplace=True)
lucro['MARGEM_ITEM_UNITARIO'] = lucro['VALOR ITEM'] - lucro['VALOR ESTOQUE']
lucro['TOTAL_MARGEM_ITEM'] = lucro['MARGEM_ITEM_UNITARIO'] * lucro['QTD ITEM']
pdf.add_table("Ranking Margem por Categoria", lucro.groupby('CATEGORIA')['TOTAL_MARGEM_ITEM'].sum().reset_index())

# Pergunta 9
compras = pd.merge(transacoes_vendas_df[['ID CLIENTE', 'ID PRODUTO']], cadastro_clientes_df, on='ID CLIENTE', how='left')
compras = pd.merge(compras, cadastro_produtos_df[['ID PRODUTO', 'NOME PRODUTO']], on='ID PRODUTO', how='left')
compras.drop_duplicates(subset=['ID CLIENTE', 'ID PRODUTO'], inplace=True)
pdf.add_table("Produtos Comprados por Cliente", compras[['NOME CLIENTE', 'NOME PRODUTO']].fillna('Desconhecido'))

# Pergunta 10
estoque_final = pd.merge(cadastro_estoque_df[['ID ESTOQUE', 'QTD ESTOQUE']], cadastro_produtos_df[['ID PRODUTO', 'ID ESTOQUE', 'NOME PRODUTO']], on='ID ESTOQUE', how='left')
estoque_final.dropna(subset=['ID PRODUTO', 'QTD ESTOQUE'], inplace=True)
est_total = estoque_final.groupby(['ID PRODUTO', 'NOME PRODUTO'])['QTD ESTOQUE'].sum().reset_index()
pdf.add_table("Ranking Produtos por Estoque", est_total)

pdf.output(output_pdf)
print(f"Relatório salvo em: {output_pdf}")


Relatório salvo em: relatorio_resultados.pdf
